In [32]:
%load_ext autoreload
%autoreload 2
import os
import pickle as pkl
from os.path import join as oj

import dvu
import matplotlib as mpl
import matplotlib.pyplot as plt
from math import ceil
from tqdm import tqdm
import numpy as np
import matplotlib.patches as patches
import sys
sys.path.append('..')

mpl.rcParams['figure.dpi'] = 250
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

# change working directory to project root
while os.getcwd().split('/')[-1] != 'imodels-experiments':
    os.chdir('..')

from config.figs_ensembles.datasets import DATASETS_CLASSIFICATION, DATASETS_REGRESSION
import viz

print('num_datasets', len(DATASETS_CLASSIFICATION), len(DATASETS_REGRESSION))

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
num_datasets 11 9


In [36]:
# run comparison aggregations
!python3 02_aggregate_results.py

  0%|                                                   | 0/100 [00:00<?, ?it/s]02_aggregate_results.py:53: UserWarning: bad complexity range
  warnings.warn(f'bad complexity range')
  4%|█▋                                         | 4/100 [00:24<09:47,  6.12s/it]/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
  5%|██▏                                        | 5/100 [00:32<10:23,  6.56s/it]/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. 

# overarching plots

In [ ]:
viz.plot_bests(metric='rocauc',
                     datasets=DATASETS_CLASSIFICATION,
                models_to_include=['Linear', 'CART', 'FIGS', 'RandomForest', 'RFFIGS-10', 'RFFIGS-10-sub50'], # FIGS, BaggingFIGS, 'RFFIGS-log2', 'RFFIGS', 'RFFIGS-10-sub75'
                config_name='figs_ensembles')

In [ ]:
viz.plot_bests(metric='r2',
                     datasets=DATASETS_REGRESSION,
                models_to_include=['Linear', 'CART', 'FIGS', 'RandomForest', 'RFFIGS-10', 'RFFIGS-10sub50'], # FIGS, 'BaggingFIGS', 'RFFIGS-log2', 'RFFIGS', 
                config_name='figs_ensembles')

In [ ]:
# df = pkl.load(open('results/saps/friedman1/train-test/results_aggregated.pkl', 'rb'))['df_mean']
df.keys()